In [1]:
import io
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "apikey.json"

from google.cloud import vision
from google.cloud.vision import types
from google.cloud import texttospeech
from google.cloud import translate
from PIL import Image

import cv2 as cv

In [2]:
class virtual_eyes:
    
    def __init__(self): #analzye_type; resim analizi için 0, yazı tanıma için 1
        self.client = vision.ImageAnnotatorClient()
        self.translate_client = translate.Client()
        self.ttsclient = texttospeech.TextToSpeechClient()
    
    def take_a_photo(self):
        try:
            cap = cv.VideoCapture(0) # 0 indexli kameradan görüntü alıyoruz
            ret, frame = cap.read() # kamera görüntüsünü alıyoruz
            
            b,g,r = cv.split(frame)
            photo = cv.merge([r,g,b]) # bir nevi renklendirme yapıyoruz
            cap.release()
            cv.imwrite('takenphoto.jpg', photo) # resim dosya olarak kaydediliyor
            return True # resim çekilip kaydedildi
            #bunun düzeltilmesi gerekiyor, resmin dosya olarak kaydedilip sonra açılıp okunması yerine
            #resmin sınıfı ve tipi, googlede kullanılacak olan tipe çevrilip döndürülmeli
            #dönüşüm işlemini araştırmak lazım
        except:
            return False # hata çıktı demektir
    
    def make_analysis(self, analyze_type):
        try:
            with io.open("takenphoto.jpg", 'rb') as image_file:
                img = image_file.read()

            image = types.Image(content=img)

            for_return = ""
            if analyze_type == 0: # resim analizi için
                list_of_findings = []
                response = self.client.label_detection(image=image)
                for label in response.label_annotations:
                    list_of_findings.append(label.description)
                # çeviri işlemleri burada yapılacak
                # önemine göre kelimeler burada listeden çıkarılabilir
                for_return = self.translate_texts(list_of_findings)
            else:
                list_of_findings = []
                response = self.client.text_detection(image=image)
                for text in response.text_annotations:
                    list_of_findings.append(text.description)
                # düzeltme işlemleri burada yapılacak
                for_return = ' '.join(list_of_findings)

            return for_return # return edilecek şey bir string değişken olacak (çıkarımlar, yada yazılar)
        except:
            return None
    
    def translate_texts(self, list_of_texts):
        text = ', '.join(list_of_texts)
        # çevirisi yapılacak dil Türkçe için tr
        target = 'tr'
        # kaynak dil
        source = 'en'
        
        translation = self.translate_client.translate(text, source_language=source, target_language=target)

        return translation['translatedText'] # çevrilen yazı cümle olarak çevrilir
    
    def text_to_speech(self, text):
        try:
            synthesis_input = texttospeech.types.SynthesisInput(text=text)
            voice = texttospeech.types.VoiceSelectionParams(
                language_code='tr-TR',
                ssml_gender=texttospeech.enums.SsmlVoiceGender.NEUTRAL)
            
            audio_config = texttospeech.types.AudioConfig(
                audio_encoding=texttospeech.enums.AudioEncoding.MP3)
            
            response = self.ttsclient.synthesize_speech(synthesis_input, voice, audio_config)
            
            # output.mp3 oluşturulacak dosya
            with open('output.mp3', 'wb') as out:
                out.write(response.audio_content)
            
            return True
        except:
            return False
            
    def display_the_speech(self, speech):
        try:
            os.startfile(speech)
            return True
        except:
            return False

In [5]:
va = virtual_eyes()
ret = va.take_a_photo()
if ret:
    returned_text = va.make_analysis(1)
    
    if returned_text != None:
        ret2 = va.text_to_speech(returned_text)
        
        if ret2:
            ret3 = va.display_the_speech('output.mp3')
            
            if ret3 == False: print("Ses Çalmada Hata") 
        else: print("Yazı Seslendirmede Hata")
    else: print("Resim Analizinde Hata")
else: print("Resim Çekmede Hata")